In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle

In [2]:
dataset = pd.read_csv("Churn_Modelling.csv",index_col=False)
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
len(dataset)

10000

In [4]:
dataset.isnull().sum()

RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

#### Preprocessing

In [5]:
### Drop irrelevant features

dataset = dataset.drop(["RowNumber","CustomerId","Surname"],axis=1)
dataset.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [6]:
### Encode categroical features

label_encoder_gender = LabelEncoder()
dataset["Gender"] = label_encoder_gender.fit_transform(dataset['Gender'])

dataset.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [7]:
### One hot encoding on nominal categorical feature, defying ordinal relationship impact on model

from sklearn.preprocessing import OneHotEncoder

encoder_geo = OneHotEncoder()

encoded_geo = encoder_geo.fit_transform(dataset[['Geography']])   ### one_hot_encoder expect 2D array(a dataframe, not 1D-series)

encoded_geo

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [8]:
encoded_feature = encoder_geo.get_feature_names_out(['Geography'])   
encoded_feature
## it gives the name for new encoded columns,so requires original column name only. Doesn't do any operation on dataset

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [9]:
encoded_geo_df = pd.DataFrame(encoded_geo.toarray(),columns = encoded_feature)
encoded_geo_df


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [10]:
dataset = pd.concat([dataset.drop(['Geography'],axis=1),encoded_geo_df],axis=1)
dataset.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [11]:
len(dataset)

10000

In [12]:
dataset.isna().sum()

CreditScore          0
Gender               0
Age                  0
Tenure               0
Balance              0
NumOfProducts        0
HasCrCard            0
IsActiveMember       0
EstimatedSalary      0
Exited               0
Geography_France     0
Geography_Germany    0
Geography_Spain      0
dtype: int64

#### Saving the encoders in a pickle file to be used as it is on test data or any new data

In [13]:
with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender,file)

with open("encoder_geography.pkl",'wb') as file:
    pickle.dump(encoder_geo,file)

In [14]:
### Divide the dataset into dependent and independent features, also train and test split

x = dataset.drop(['Exited'],axis=1)
y = dataset["Exited"]

x,y

(      CreditScore  Gender  Age  Tenure    Balance  NumOfProducts  HasCrCard  \
 0             619       0   42       2       0.00              1          1   
 1             608       0   41       1   83807.86              1          0   
 2             502       0   42       8  159660.80              3          1   
 3             699       0   39       1       0.00              2          0   
 4             850       0   43       2  125510.82              1          1   
 ...           ...     ...  ...     ...        ...            ...        ...   
 9995          771       1   39       5       0.00              2          1   
 9996          516       1   35      10   57369.61              1          1   
 9997          709       0   36       7       0.00              1          0   
 9998          772       1   42       3   75075.31              2          1   
 9999          792       0   28       4  130142.79              1          1   
 
       IsActiveMember  EstimatedSalary

In [15]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=12)

#### Scaling for Uniformity

In [16]:
scaler = StandardScaler() ### creating a object of the class

scaler.fit(x_train)

StandardScaler()

In [17]:
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

In [18]:
### pickling Standard Scaler

with open("standard_scaler.pkl","wb") as file:
    pickle.dump(scaler,file)

### Training ANN Model

In [19]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

In [20]:
(x_train.shape[1],)

(12,)

In [21]:
### Building ANN architecture

model = Sequential([
                    Dense(64,activation="relu",input_shape = (x_train.shape[1],)),   ### ---> Hidden Layer 1, ALso provide shape of input layer so that the HL1 knows how many input neurons will be connected 
                    Dense(32,activation='relu'),   ### ---> Hidden Layer 2
                    Dense(1,activation="sigmoid")   ### ---> output layer
])

c:\Users\ViditAgarwal\Desktop\AI ML learned code\Deep Learning\ANN Churn Modelling\venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [22]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [23]:
### Compile the model


optimizer_func = tf.keras.optimizers.Adam(learning_rate = 0.05)

loss_func = tf.keras.losses.BinaryCrossentropy()

model.compile(optimizer=optimizer_func,loss=loss_func,metrics=["accuracy"])

### Another way to compile:

 ### model.compile(optimizer="adam",loss="binary_crossentropy",metrics = ["accuracy"])

In [24]:
### Setting Up TensorBoard

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

### a log directory is created, where all the the logs will be saved and thus we can access the log to analyze the model's performance.

tensorflow_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

- TensorBoard lets log the training of the model each time we run the training dataset. 
- It is a visualization board

In [25]:
### Setting up EarlyStopping

early_stopping_callback = EarlyStopping(monitor='val_loss',patience =10,restore_best_weights=True)


### basically the idea is that for each epochs the early stopping will monitor validation loss specifically and when it sees negligible change 
### it will wait until next 5 epochs, and then restore the best weights which it can find.

- Early Stopping callback is helpful to stop the training epochs when the loss change is negligble.
- It saves computational resources and time

### Training the model

In [26]:
history = model.fit(
    x_train,y_train,validation_data = (x_test,y_test),epochs=100,
    callbacks = [tensorflow_callback,early_stopping_callback]
    )

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.8183 - loss: 0.4595 - val_accuracy: 0.8520 - val_loss: 0.3887
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8484 - loss: 0.3741 - val_accuracy: 0.8505 - val_loss: 0.4025
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8493 - loss: 0.3700 - val_accuracy: 0.8475 - val_loss: 0.3735
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8554 - loss: 0.3590 - val_accuracy: 0.8415 - val_loss: 0.3802
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8522 - loss: 0.3589 - val_accuracy: 0.8490 - val_loss: 0.3744
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.8493 - loss: 0.3688 - val_accuracy: 0.8470 - val_loss: 0.3742
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.8579 - loss: 0.3553 - val_accuracy: 0.8580 - val_loss: 0.3654
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8577 - loss: 0.3537 - val_accu

In [27]:
model.save("model.h5")  ## .h5 is compatible with kears api

In [28]:
### Loading TensorBoard Extension

%load_ext tensorboard

In [30]:
%tensorboard --logdir logs/fit/20250222-143419



Reusing TensorBoard on port 6006 (pid 24272), started 0:00:07 ago. (Use '!kill 24272' to kill it.)

#### Load the model

- Go to new ipynb file -- check there how it is done